### 1. 讀入使用套件

第一輪是讀進我們基本套件, 第二輪是 TensorFlow 用到的套件。

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import pickle

from tensorflow.keras.models import load_model, model_from_json

### 2. 讀入訓練好的 RNN 紅樓夢生成器

這裡是參考[《精通機器學習-使用 Scikit-Learn, Keras 與 TensorFlow》](https://www.books.com.tw/products/0010854043?sloc=main)這本書中莎士比亞生成器的部份寫成的。架構很簡單,就每輸入 100 個字, 預測下一個字是什麼。雙層 LSTM, 每層 128 個神經元。訓練 10 次, 在 1080Ti GPU 的電腦上大概花了 10 個小時。

以下會從我的 GitHub 重新讀入模型, 已下載模型請直接跳到下一段。


In [ ]:
from urllib.request import urlretrieve

In [ ]:
urlretrieve("https://raw.githubusercontent.com/yenlung/Deep-Learning-Basics/master/dream_rnn_architecture.json", "architecture.json")
urlretrieve("https://github.com/yenlung/Deep-Learning-Basics/raw/master/dream_rnn_weights.h5", "weights.h5")
urlretrieve("https://github.com/yenlung/Deep-Learning-Basics/raw/master/dream_tokenizer2.pkl", "tokenizer.pkl")

('tokenizer.pkl', <http.client.HTTPMessage at 0x7f09216c9990>)

In [ ]:
f = open('architecture.json', 'r')
loaded_model = f.read()
f.close()

In [ ]:
model = model_from_json(loaded_model)

In [ ]:
model.load_weights("weights.h5")

In [ ]:
f = open('tokenizer.pkl', 'rb')
tokenizer = pickle.load(f)
f.close()

#### 【已下載紅樓夢生成器請執行這段】

如果你已經在[我的 GitHub](https://github.com/yenlung/Deep-Learning-Basics) 中下載紅樓夢生成模型, 這包括:

1. `dream_rnn` 資料夾, 這是模型和訓練好的權重。
2. `dream_tokenizer2.pkl` 檔案, 這是模型使用的 tokenizer。

存到你的 Google Drive, 放 Colab 程式的地方 (預設是 `Colab Notebooks` 資料夾)。那執行下面這一段, 不用再讀入一次模型資料。

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
model = load_model('dream_rnn')

In [ ]:
f = open('dream_tokenizer2.pkl', 'rb')
tokenizer = pickle.load(f)
f.close()

### 4. 製造紅樓夢生成器

首先 `max_id` 是記錄《紅樓夢》用到的所有不同的中文字字數, 包括新式標點符號。很讓人驚訝 (?) 的是, 字數並沒有想像中多。

In [ ]:
max_id = len(tokenizer.word_index)

接下來是一段文字, 我們用事先訓練好的 tokenizer 換成一段數字, 最後用 one-hot encoding 回傳。

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences([texts]))-1
    return tf.one_hot(X, max_id)

這段程式主要依輸入的一段文字, 用我們的 model 去預測下一個字。注意像平常的分類問題, 這裡輸出是每人個字出現機率最高的。但都照這樣, 我們輸入同一段文字, 之後出現的文字永遠是一樣的! 常用的手法是去設定 `temperature`, `temperature` 接近 0, 大致上就取機率最高的字; `temperature` 越大就越隨機。太隨機就變成亂數取字! 一般 `temperature`設 1 左右效果最佳。

In [ ]:
def next_char(texts, temperature=1):
    X_new = preprocess(texts)
    y_predict = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_predict) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

最後就一段文字進來，再產生 `n_chars` 這麼多個字。我們原本一段文字只能生一個字, 那就一次生一個字, 最後要生多少個字就生多少個字。

原本訓練我們一段是 100 個字去訓練的, 這裡超過 100 字時我們就取最後 100 個字丟入模型。

In [ ]:
def complete_text(texts, n_chars=50, temperature=1):
    n_chars=int(n_chars)
    for _ in range(n_chars):
        texts = texts + next_char(texts[-100:], temperature)
    return texts

做成 web app 前, 先來測試一下。

In [ ]:
complete_text("自孫悟空從石頭中蹦出來之後，", n_chars=300, temperature=0.2)

'自孫悟空從石頭中蹦出來之後，說道：「寶玉既有此事，不知何俗緣？」士隱道：「神仙長得如此。」雨村聽了，益發驚異：「請問仙長，何必如此？」士隱道：「寶玉，何嘗肯唸書，你這條心，原是如此的情緣完結，也不知道了。」雨村聽了，益發驚異：「你們這麼一個人，不知何俗而不！」雨村聽了，益發驚異：「請問仙長，何必如此？」士隱道：「此事不知。」雨村聽了，益發驚異：「請問仙長，何必如此？」士隱道：「此事不知。」雨村聽了，益發驚異：「請問仙長，何必如此？」士隱道：「神仙長佛。」雨村聽了，益發驚異：「請問仙長，何必如此？」士隱道：「神仙長名久，無佛有所。」雨村聽了，益發驚異：「你們不知道士隱，雨村再不知道。」雨村聽了，益發驚異：「請問仙長，何必如'

### 5. 用 `gradio` 做成一個網路 app

我們準備用 [`gradio`](https://gradio.app/) 套件, 神速做完一個 web app。最酷的是, 最後出現 `https://xxxx.gradio.app` 那個網址, 在你的 Colab 還在執行的時候, 任何人都可以用任何瀏覽器連進來使用!

In [ ]:
!pip install gradio

     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 1.9MB 35.5MB/s 
     |████████████████████████████████| 215kB 51.4MB/s 
     |████████████████████████████████| 3.2MB 47.0MB/s 
     |████████████████████████████████| 962kB 50.9MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-cp37-none-any.whl size=3372 sha256=f461cda770763f9a22d858136f2f8668864552af66767ad7f1c63c401d626f16
  Stored in directory: /root/.cache/pip/wheels/9f/fc/a7/ab5712c3ace9a8f97276465cc2937316ab8063c1fea488ea77
  Created wheel for ffmpy: filename=ffmpy-0.3.0-cp37-none-any.whl size=4710 sha256=f898d0bb99821a1d92f3d3d3a261c0b937b3c4a4ea2d6201bcf348d05c70084f
  Stored in directory: /root/.cache/pip/wheels/cc/ac/c4/bef572cb7e52bfca170046f567e64858632daf77e0f34e5a74
Successfully built flask-cachebuster ffmpy


In [ ]:
import gradio as gr

In [ ]:
iface = gr.Interface(
    fn=complete_text,
    inputs=[
        "text",
        gr.inputs.Slider(50, 200, 1, 50),
        gr.inputs.Slider(0.2, 2, 0.2, 1)],
    outputs="text",
    title="紅樓夢生成器",
    description="起個頭, 幫你完成一段紅樓夢。可以改變 temperature, 越小生出的字越固定, 越大越隨機。")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://47961.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://47961.gradio.app')